# __Analisis de Evasion de Clientes__

## Librerias

In [1]:
import pandas as pd
import numpy as np

## Extraccion de datos

### Importar Datos

In [2]:
url = 'https://raw.githubusercontent.com/ingridcristh/challenge2-data-science-LATAM/refs/heads/main/TelecomX_Data.json'

df = pd.read_json(url)
df.head()

,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
3,0011-IGKFF,Yes,"{'gender': 'Male', 'SeniorCitizen': 1, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
4,0013-EXCHZ,Yes,"{'gender': 'Female', 'SeniorCitizen': 1, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."


Como vemos, hay varias variables que contienen diccionarios en sus valores. Por tanto, debemos expandirlas

In [3]:
# expandir cada variable
df_customer = pd.json_normalize(df['customer'])
df_phone = pd.json_normalize(df['phone'])
df_internet = pd.json_normalize(df['internet'])
df_account = pd.json_normalize(df['account'])

In [4]:
# seleccionar variables no expandidas de df
df = df[['customerID', 'Churn']]

In [5]:
# unir todos los dfs en uno
df_exp = pd.concat([df, df_customer, df_phone, df_internet, df_account], axis=1)
df_exp.head(5)

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.4


### Contexto de los datos

Se tratan de datos de los clientes de una empresa cuya informacion de cada variable se interpreta como:

- _customerID_: número de identificación único de cada cliente
- _Churn_: si el cliente dejó o no la empresa
- _gender_: género (masculino y femenino)
- _SeniorCitizen_: información sobre si un cliente tiene o no una edad igual o mayor a 65 años
- _Partner_: si el cliente tiene o no una pareja
- _Dependents_: si el cliente tiene o no dependientes
- _tenure_: meses de contrato del cliente
- _PhoneService_: suscripción al servicio telefónico
- _MultipleLines_: suscripción a más de una línea telefónica
- _InternetService_: suscripción a un proveedor de internet
- _OnlineSecurity_: suscripción adicional de seguridad en línea
- _OnlineBackup_: suscripción adicional de respaldo en línea
- _DeviceProtection_: suscripción adicional de protección del dispositivo
- _TechSupport_: suscripción adicional de soporte técnico, menor tiempo de espera
- _StreamingTV_: suscripción de televisión por cable
- _StreamingMovies_: suscripción de streaming de películas
- _Contract_: tipo de contrato
- _PaperlessBilling_: si el cliente prefiere recibir la factura en línea
- _PaymentMethod_: forma de pago
- _Charges.Monthly_: total de todos los servicios del cliente por mes
- _Charges.Total_: total gastado por el cliente

### Tipo de datos

In [6]:
df_exp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7267 non-null   object 
 1   Churn             7267 non-null   object 
 2   gender            7267 non-null   object 
 3   SeniorCitizen     7267 non-null   int64  
 4   Partner           7267 non-null   object 
 5   Dependents        7267 non-null   object 
 6   tenure            7267 non-null   int64  
 7   PhoneService      7267 non-null   object 
 8   MultipleLines     7267 non-null   object 
 9   InternetService   7267 non-null   object 
 10  OnlineSecurity    7267 non-null   object 
 11  OnlineBackup      7267 non-null   object 
 12  DeviceProtection  7267 non-null   object 
 13  TechSupport       7267 non-null   object 
 14  StreamingTV       7267 non-null   object 
 15  StreamingMovies   7267 non-null   object 
 16  Contract          7267 non-null   object 


> El conjunto de datos esta conformado por 7267 registros y 21 variables.

> Todas las variables tienen 7267 registros lo que indica la ausencia de valores faltantes.

> Las variables 'SeniorCitizen' y 'tenure' son de tipo entero, la variable 'Charges.Monthly' es de tipo float y el resto de variables son de tipo object.

> La variable 'Charges.Total' y 'Churn' en la practica son de tipo numerico. Sin embargo, Pandas las detecta de tipo object lo que sugiere un problema con la consistencia de los datos.

## Consistencia de los datos

### Registros duplicados

In [7]:
# verificar registros duplicados
df_exp.duplicated().sum()

0

> No hay registros duplicados en el dataframe.

### Tratar valores vacios

Verificamos los valores de 'Churn' y 'Charges.Total'

In [8]:
print('Charges.Total: ',df_exp['Charges.Total'].unique())
print('Churn: ',df_exp.Churn.unique())

Charges.Total:  ['593.3' '542.4' '280.85' ... '742.9' '4627.65' '3707.6']
Churn:  ['No' 'Yes' '']


> Los valores de _Churn_ y _Charges.Total_ presentan valores vacios por lo que procedemos a tratarlos

__Churn__

In [9]:
# registros con valores vacios en _Churn_
df_exp[df_exp['Churn'] == '']

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total
30,0047-ZHDTW,,Female,0,No,No,11,Yes,Yes,Fiber optic,...,No,No,No,No,No,Month-to-month,Yes,Bank transfer (automatic),79.00,929.3
75,0120-YZLQA,,Male,0,No,No,71,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,Yes,Credit card (automatic),19.90,1355.1
96,0154-QYHJU,,Male,0,No,No,29,Yes,No,DSL,...,Yes,No,Yes,No,No,One year,Yes,Electronic check,58.75,1696.2
98,0162-RZGMZ,,Female,1,No,No,5,Yes,No,DSL,...,Yes,No,Yes,No,No,Month-to-month,No,Credit card (automatic),59.90,287.85
175,0274-VVQOQ,,Male,1,Yes,No,65,Yes,Yes,Fiber optic,...,Yes,Yes,No,Yes,Yes,One year,Yes,Bank transfer (automatic),103.15,6792.45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7158,9840-GSRFX,,Female,0,No,No,14,Yes,Yes,DSL,...,Yes,No,No,No,No,One year,Yes,Mailed check,54.25,773.2
7180,9872-RZQQB,,Female,0,Yes,No,49,No,No phone service,DSL,...,No,No,No,Yes,No,Month-to-month,No,Bank transfer (automatic),40.65,2070.75
7211,9920-GNDMB,,Male,0,No,No,9,Yes,Yes,Fiber optic,...,No,No,No,No,No,Month-to-month,Yes,Electronic check,76.25,684.85
7239,9955-RVWSC,,Female,0,Yes,Yes,67,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,Yes,Bank transfer (automatic),19.25,1372.9


> Hay 224 registros con valores vacios en _Churn_ lo que representa el 3% de los registros totales. Como la variable objetivo es _Churn_, sin esta informacion tales registros no aportan al analisis por tanto los eliminamos.

In [10]:
# elimina registros con Churn vacios
df_exp = df_exp[df_exp['Churn'] != '']
df_exp.shape

(7043, 21)

In [11]:
# verificar que Churn este libre de valores vacios
print(df_exp.Churn.unique())

['No' 'Yes']


__Charges.Total__

In [12]:
# registros vacios en Charges.Total
df_exp[df_exp['Charges.Total'] == ' ']

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total
975,1371-DWPAZ,No,Female,0,Yes,Yes,0,No,No phone service,DSL,...,Yes,Yes,Yes,Yes,No,Two year,No,Credit card (automatic),56.05,
1775,2520-SGTTA,No,Female,0,Yes,Yes,0,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.00,
1955,2775-SEFEE,No,Male,0,No,Yes,0,Yes,Yes,DSL,...,Yes,No,Yes,No,No,Two year,Yes,Bank transfer (automatic),61.90,
2075,2923-ARZLG,No,Male,0,Yes,Yes,0,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,One year,Yes,Mailed check,19.70,
2232,3115-CZMZD,No,Male,0,No,Yes,0,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.25,
2308,3213-VVOLG,No,Male,0,Yes,Yes,0,Yes,Yes,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.35,
2930,4075-WKNIU,No,Female,0,Yes,Yes,0,Yes,Yes,DSL,...,Yes,Yes,Yes,Yes,No,Two year,No,Mailed check,73.35,
3134,4367-NUYAO,No,Male,0,Yes,Yes,0,Yes,Yes,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.75,
3203,4472-LVYGI,No,Female,0,Yes,Yes,0,No,No phone service,DSL,...,No,Yes,Yes,Yes,No,Two year,Yes,Bank transfer (automatic),52.55,
4169,5709-LVOEQ,No,Female,0,Yes,Yes,0,Yes,No,DSL,...,Yes,Yes,No,Yes,Yes,Two year,No,Mailed check,80.85,


> Hay 11 registros con valores vacios en Charges.Total lo que representa el 0.1% de los registros totales. Al visualizarlos, nos damos cuenta que corresponden a clientes con contratos largos pero que no tienen ni el primer mes cumplido por lo que no aportan informacion relevante al estudio. Los eliminamos

In [13]:
# elimina registros con Charges.Total vacios
df_exp = df_exp[df_exp['Charges.Total'] != ' ']
df_exp.shape

(7032, 21)

In [14]:
# verificar que Churn este libre de valores vacios
print(df_exp['Charges.Total'].unique())

['593.3' '542.4' '280.85' ... '742.9' '4627.65' '3707.6']


In [15]:
import numpy as np 

# convertir a tipo entero
df_exp['Charges.Total'] = df_exp['Charges.Total'].astype(np.float64)

# verificar tipo
df_exp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7032 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7032 non-null   object 
 1   Churn             7032 non-null   object 
 2   gender            7032 non-null   object 
 3   SeniorCitizen     7032 non-null   int64  
 4   Partner           7032 non-null   object 
 5   Dependents        7032 non-null   object 
 6   tenure            7032 non-null   int64  
 7   PhoneService      7032 non-null   object 
 8   MultipleLines     7032 non-null   object 
 9   InternetService   7032 non-null   object 
 10  OnlineSecurity    7032 non-null   object 
 11  OnlineBackup      7032 non-null   object 
 12  DeviceProtection  7032 non-null   object 
 13  TechSupport       7032 non-null   object 
 14  StreamingTV       7032 non-null   object 
 15  StreamingMovies   7032 non-null   object 
 16  Contract          7032 non-null   object 


### Columna de cuentas diarias

In [16]:
df_exp

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.60,593.30
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.90,542.40
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.90,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.00,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.90,267.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7262,9987-LUTYD,No,Female,0,No,No,13,Yes,No,DSL,...,No,No,Yes,No,No,One year,No,Mailed check,55.15,742.90
7263,9992-RRAMN,Yes,Male,0,Yes,No,22,Yes,Yes,Fiber optic,...,No,No,No,No,Yes,Month-to-month,Yes,Electronic check,85.10,1873.70
7264,9992-UJOEL,No,Male,0,No,No,2,Yes,No,DSL,...,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,50.30,92.75
7265,9993-LHIEB,No,Male,0,Yes,Yes,67,Yes,No,DSL,...,No,Yes,Yes,No,Yes,Two year,No,Mailed check,67.85,4627.65


In [17]:
# crear columna de gasto diario de cada cliente
df_exp['Cuentas_Diarias'] = (df_exp['Charges.Monthly'])/(df_exp['tenure']*30)
df_exp.head()

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total,Cuentas_Diarias
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.30,0.242963
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.40,0.221852
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85,0.615833
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85,0.251282
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.40,0.932222


### Estandarizacion de los datos

Vamos a convertir los datos de las variables tipo object a enteros binarios donde sea posible. Verifiquemos los valores unicos de cada variable.

In [18]:
# verificar valores de variables de tipo object
print('gender: ',df_exp.gender.unique())
print('SeniorCitizen: ',df_exp.SeniorCitizen.unique())
print('Partner: ',df_exp.Partner.unique())
print('Dependents: ',df_exp.Dependents.unique())
print('PhoneService: ',df_exp.PhoneService.unique())
print('MultipleLines: ',df_exp.MultipleLines.unique())
print('InternetService: ',df_exp.InternetService.unique())
print('OnlineSecurity: ',df_exp.OnlineSecurity.unique())
print('OnlineBackup: ',df_exp.OnlineBackup.unique())
print('DeviceProtection: ',df_exp.DeviceProtection.unique())
print('TechSupport: ',df_exp.TechSupport.unique())
print('StreamingTV: ',df_exp.StreamingTV.unique())
print('StreamingMovies: ',df_exp.StreamingMovies.unique())
print('Contract: ',df_exp.Contract.unique())
print('PaperlessBilling: ',df_exp.PaperlessBilling.unique())
print('PaymentMethod: ',df_exp.PaymentMethod.unique())
print('Charges.Total: ',df_exp['Charges.Total'].unique())
print('Churn: ',df_exp.Churn.unique())

gender:  ['Female' 'Male']
SeniorCitizen:  [0 1]
Partner:  ['Yes' 'No']
Dependents:  ['Yes' 'No']
PhoneService:  ['Yes' 'No']
MultipleLines:  ['No' 'Yes' 'No phone service']
InternetService:  ['DSL' 'Fiber optic' 'No']
OnlineSecurity:  ['No' 'Yes' 'No internet service']
OnlineBackup:  ['Yes' 'No' 'No internet service']
DeviceProtection:  ['No' 'Yes' 'No internet service']
TechSupport:  ['Yes' 'No' 'No internet service']
StreamingTV:  ['Yes' 'No' 'No internet service']
StreamingMovies:  ['No' 'Yes' 'No internet service']
Contract:  ['One year' 'Month-to-month' 'Two year']
PaperlessBilling:  ['Yes' 'No']
PaymentMethod:  ['Mailed check' 'Electronic check' 'Credit card (automatic)'
 'Bank transfer (automatic)']
Charges.Total:  [ 593.3   542.4   280.85 ...  742.9  4627.65 3707.6 ]
Churn:  ['No' 'Yes']


__Gender__

In [19]:
# convertir valores de genero a binario
# 0:mujer. 1:hombre
df_exp['gender'] = df_exp['gender'].apply(lambda x: x.replace('Female', '0').replace('Male', '1')).astype(np.int64)
df_exp['gender'][:5] 

0    0
1    1
2    1
3    1
4    0
Name: gender, dtype: int64

__Churn__

In [20]:
# convertir valores de churn a binario
# 0:No esta con la empresa. 1: Sigue en la empresa
df_exp['Churn'] = df_exp['Churn'].replace({'Yes':'1', 'No':'0'}).astype(np.int64)
df_exp['Churn'][:5]

0    0
1    0
2    1
3    1
4    1
Name: Churn, dtype: int64

__Partner, Dependents, PhoneService y PaperlessBilling__

In [21]:
# lista de variables a convertir en binario
lista = ['Partner', 'Dependents', 'PhoneService', 'PaperlessBilling']

df_exp[lista] = df_exp[lista].apply(lambda x: x.replace('Yes', '1').replace('No', '0')).astype(np.int64)
df_exp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7032 entries, 0 to 7266
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7032 non-null   object 
 1   Churn             7032 non-null   int64  
 2   gender            7032 non-null   int64  
 3   SeniorCitizen     7032 non-null   int64  
 4   Partner           7032 non-null   int64  
 5   Dependents        7032 non-null   int64  
 6   tenure            7032 non-null   int64  
 7   PhoneService      7032 non-null   int64  
 8   MultipleLines     7032 non-null   object 
 9   InternetService   7032 non-null   object 
 10  OnlineSecurity    7032 non-null   object 
 11  OnlineBackup      7032 non-null   object 
 12  DeviceProtection  7032 non-null   object 
 13  TechSupport       7032 non-null   object 
 14  StreamingTV       7032 non-null   object 
 15  StreamingMovies   7032 non-null   object 
 16  Contract          7032 non-null   object 


- __Partner:__ 1 si tiene pareja, 0 en caso contrario.

- __Dependents:__ 1 si tiene personas que dependen de él, 0 en caso contrario.

- __PhoneService:__ 1 si esta suscrito al servicio telefonico, 0 en caso contrario.

- __PaperlessBilling:__ 1 si prefiere recibir la factura en linea, 0 en caso contrario.

Analicemos las variables que presentan mas de dos opciones: MultipleLines, InternetService, OnlineSecurity, OnlineBackup, DeviceProtection, TechSupport, StreamingTV, StreamingMovies y Contract

__MultipleLines__

In [22]:
# analisis MultipleLines
df_exp[df_exp['PhoneService'] == 1].head()

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total,Cuentas_Diarias
0,0002-ORFBO,0,0,0,1,1,9,1,No,DSL,...,No,Yes,Yes,No,One year,1,Mailed check,65.6,593.30,0.242963
1,0003-MKNFE,0,1,0,0,0,9,1,Yes,DSL,...,No,No,No,Yes,Month-to-month,0,Mailed check,59.9,542.40,0.221852
2,0004-TLHLJ,1,1,0,0,0,4,1,No,Fiber optic,...,Yes,No,No,No,Month-to-month,1,Electronic check,73.9,280.85,0.615833
3,0011-IGKFF,1,1,1,1,0,13,1,No,Fiber optic,...,Yes,No,Yes,Yes,Month-to-month,1,Electronic check,98.0,1237.85,0.251282
4,0013-EXCHZ,1,0,1,1,0,3,1,No,Fiber optic,...,No,Yes,Yes,No,Month-to-month,1,Mailed check,83.9,267.40,0.932222


In [23]:
df_exp[df_exp['PhoneService'] == 0].head()

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total,Cuentas_Diarias
10,0017-DINOC,0,1,0,0,0,54,0,No phone service,DSL,...,No,Yes,Yes,No,Two year,0,Credit card (automatic),45.20,2460.55,0.027901
14,0019-GFNTW,0,0,0,0,0,56,0,No phone service,DSL,...,Yes,Yes,No,No,Two year,0,Bank transfer (automatic),45.05,2560.10,0.026815
16,0020-JDNXP,0,0,0,1,1,34,0,No phone service,DSL,...,Yes,Yes,Yes,Yes,One year,0,Mailed check,61.25,1993.20,0.060049
19,0023-HGHWL,1,1,1,0,0,1,0,No phone service,DSL,...,No,No,No,No,Month-to-month,1,Electronic check,25.10,25.10,0.836667
25,0032-PGELS,1,0,0,1,1,1,0,No phone service,DSL,...,No,No,No,No,Month-to-month,0,Bank transfer (automatic),30.50,30.50,1.016667


> Observamos que los clientes que estan suscrito al servicio telefonico necesariamente tienen al menos 1 linea. Los que no estan suscrito, no tienen linea telefonica. Por tanto, esta variable no nos brinda informacion relevante, solo informacion redundante de 'PhoneService'. La omitimos del analisis.

In [24]:
df_exp = df_exp.drop('MultipleLines', axis=1)

Tambien omitimos del analisis la variable customer_ID ya que son identificadores de los clientes que a nivel general no aporta informacion relevante.

In [25]:
df_exp = df_exp.drop('customerID', axis=1)

__InternetService__

Analizamos la variable InternetService. Presenta 3 valores: 'DSL' 'Fiber optic' y 'No'. Podemos simplificar las opciones a 2 variables binarias: 1 si esta suscrito al servicio de internet bien sea de tipo DSL o Fibra Optica y 0 de lo contrario.

In [26]:
df_exp['InternetService'] = df_exp['InternetService'].replace({'DSL':'1', 'Fiber optic':'1', 'No':'0'}).astype(np.int64)
df_exp['InternetService'][:5]

0    1
1    1
2    1
3    1
4    1
Name: InternetService, dtype: int64

__OnlineSecurity__

Analizamos la variable OnlineSecurity. Presenta los siguientes valores: 'No' 'Yes' y 'No internet service'. Verifiquemos los clientes que no tienen servicio de internet

In [27]:
df_exp[df_exp['InternetService'] == 1]['OnlineSecurity']

0        No
1        No
2        No
3        No
4        No
       ... 
7262    Yes
7263     No
7264     No
7265    Yes
7266    Yes
Name: OnlineSecurity, Length: 5512, dtype: object

Para este caso, hay clientes que a pesar de contar con servicio de internet no se suscriben a seguridad en linea. Podria agregar informacion valiosa al analisis por tanto, la convertimos a enteros:

- 0, no tiene servicio de internet

- 1, no tiene seguridad en linea

- 2, tiene seguridad en linea

In [28]:
df_exp['OnlineSecurity'] = df_exp['OnlineSecurity'].replace({'No internet service': 0, 'No': 1, 'Yes': 2}).astype(np.int64)
df_exp['OnlineSecurity'][:5]

0    1
1    1
2    1
3    1
4    1
Name: OnlineSecurity, dtype: int64

__OnlineBackup, DeviceProtection, TechSupport, StreamingTV, StreamingMovies, Contract__

Podemos tratar estas variables como en el caso anterior ya que presentan 3 valores:

- __OnlineBackup__: 0 no tiene servicio de internet, 1 no tiene servicio de respaldo en linea y 2 si tiene servicio de respaldo.

- __DeviceProtection__: 0 no tiene servicio de internet, 1 no tiene proteccion al dispositivo y 2 si tiene preoteccion.

- __TechSupport__: 0 no tiene servicio de internet, 1 no tiene soporte tecnico mejorado y 2 si tiene soporte tecnico.

- __StreamingTV__: 0 no tiene servicio de internet, 1 no  tiene TV por cable y 2 si tiene.

- __StreamingMovies__: 0 no tiene servicio de internet, 1 no tiene streaming a peliculas y 2 si tiene.

- __Contract__: 0 si el contrato es 'Month-to-month', 1 si es de 'One year' y 2 si es de 'Two year'.

In [29]:
df_exp['OnlineBackup'] = df_exp['OnlineBackup'].replace({'No internet service': 0, 'No': 1, 'Yes': 2}).astype(np.int64)
df_exp['DeviceProtection'] = df_exp['DeviceProtection'].replace({'No internet service': 0, 'No': 1, 'Yes': 2}).astype(np.int64)
df_exp['TechSupport'] = df_exp['TechSupport'].replace({'No internet service': 0, 'No': 1, 'Yes': 2}).astype(np.int64)
df_exp['StreamingTV'] = df_exp['StreamingTV'].replace({'No internet service': 0, 'No': 1, 'Yes': 2}).astype(np.int64)
df_exp['StreamingMovies'] = df_exp['StreamingMovies'].replace({'No internet service': 0, 'No': 1, 'Yes': 2}).astype(np.int64)
df_exp['Contract'] = df_exp['Contract'].replace({'Month-to-month': 0, 'One year': 1, 'Two year': 2}).astype(np.int64)

df_exp[['OnlineSecurity', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract']]

,OnlineSecurity,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract
0,1,1,2,2,1,1
1,1,1,1,1,2,0
2,1,2,1,1,1,0
3,1,2,1,2,2,0
4,1,1,2,2,1,0
...,...,...,...,...,...,...
7262,2,1,2,1,1,1
7263,1,1,1,1,2,0
7264,1,1,1,1,1,0
7265,2,2,2,1,2,2


### Cambiar nombre de columnas y organizarlas

In [30]:
df_exp.columns

Index(['Churn', 'gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure',
       'PhoneService', 'InternetService', 'OnlineSecurity', 'OnlineBackup',
       'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies',
       'Contract', 'PaperlessBilling', 'PaymentMethod', 'Charges.Monthly',
       'Charges.Total', 'Cuentas_Diarias'],
      dtype='object')

In [31]:
nombres_nuevos = {
    'Churn': 'cancelacion',
    'gender': 'genero',
    'SeniorCitizen': 'jubilado',
    'Partner': 'en_pareja',
    'Dependents': 'dependientes',
    'tenure': 'meses_contrato',
    'PhoneService': 'servicio_telefonico',
    'InternetService': 'servicio_internet',
    'OnlineSecurity': 'seguridad_online',
    'OnlineBackup': 'respaldo_online',
    'DeviceProtection': 'proteccion_dispositivo',
    'TechSupport': 'soporte_efectivo',
    'StreamingTV': 'Streaming_TV',
    'StreamingMovies': 'Streaming_peliculas',
    'Contract': 'tipo_contrato',
    'PaperlessBilling': 'facturacion_electronica',
    'Charges.Monthly': 'cuenta_mensual',
    'Charges.Total': 'cuentas_total',
    'PaymentMethod': 'metodo_pago',
    'Cuentas_Diarias': 'cuentas_diarias'
}

df_exp.rename(columns=nombres_nuevos, inplace=True)

In [32]:
df_exp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7032 entries, 0 to 7266
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   cancelacion              7032 non-null   int64  
 1   genero                   7032 non-null   int64  
 2   jubilado                 7032 non-null   int64  
 3   en_pareja                7032 non-null   int64  
 4   dependientes             7032 non-null   int64  
 5   meses_contrato           7032 non-null   int64  
 6   servicio_telefonico      7032 non-null   int64  
 7   servicio_internet        7032 non-null   int64  
 8   seguridad_online         7032 non-null   int64  
 9   respaldo_online          7032 non-null   int64  
 10  proteccion_dispositivo   7032 non-null   int64  
 11  soporte_efectivo         7032 non-null   int64  
 12  Streaming_TV             7032 non-null   int64  
 13  Streaming_peliculas      7032 non-null   int64  
 14  tipo_contrato           

## Carga y Analisis

Vamos a separar los datos categoricos (binarios, ternarios, strings) de los datos numericos para obtener una estadistica descriptiva mas efectiva.

In [33]:
# datarame con variables categoricas
var_bin = ['cancelacion', 'genero', 'jubilado', 'en_pareja', 'dependientes', 'servicio_telefonico', 'servicio_internet', 'seguridad_online', 'respaldo_online', 'proteccion_dispositivo', 'soporte_efectivo', 'Streaming_TV', 'Streaming_peliculas', 'tipo_contrato', 'facturacion_electronica', 'metodo_pago']
df_bin = df_exp[var_bin]
df_bin.head()

,cancelacion,genero,jubilado,en_pareja,dependientes,servicio_telefonico,servicio_internet,seguridad_online,respaldo_online,proteccion_dispositivo,soporte_efectivo,Streaming_TV,Streaming_peliculas,tipo_contrato,facturacion_electronica,metodo_pago
0,0,0,0,1,1,1,1,1,2,1,2,2,1,1,1,Mailed check
1,0,1,0,0,0,1,1,1,1,1,1,1,2,0,0,Mailed check
2,1,1,0,0,0,1,1,1,1,2,1,1,1,0,1,Electronic check
3,1,1,1,1,0,1,1,1,2,2,1,2,2,0,1,Electronic check
4,1,0,1,1,0,1,1,1,1,1,2,2,1,0,1,Mailed check


In [34]:
# dataframe con variables numericas
df_num = df_exp[[col for col in df_exp.columns if col not in var_bin]].copy()
df_num['cancelacion'] = df_exp['cancelacion']
df_num.head()

,meses_contrato,cuenta_mensual,cuentas_total,cuentas_diarias,cancelacion
0,9,65.6,593.30,0.242963,0
1,9,59.9,542.40,0.221852,0
2,4,73.9,280.85,0.615833,1
3,13,98.0,1237.85,0.251282,1
4,3,83.9,267.40,0.932222,1


### Analisis estadistico variables numericas

Analicemos las variables numericas de aquellos clientes que mantienen el servicio con la empresa.

In [35]:
# estadistica de clientes que mantienen el servicio
df_num[df_num['cancelacion'] == 0].describe()

,meses_contrato,cuenta_mensual,cuentas_total,cuentas_diarias,cancelacion
count,5163.00000,5163.000000,5163.000000,5163.000000,5163.0
mean,37.65001,61.307408,2555.344141,0.160736,0.0
std,24.07694,31.094557,2329.456984,0.328574,0.0
min,1.00000,18.250000,18.800000,0.008935,0.0
25%,15.00000,25.100000,577.825000,0.036449,0.0
50%,38.00000,64.450000,1683.600000,0.055444,0.0
75%,61.00000,88.475000,4264.125000,0.114517,0.0
max,72.00000,118.750000,8672.450000,3.195000,0.0


> La mitad de los clientes que mantienen el servicio tienen 38 meses de contrato (mas de 3 años).

> La mitad de los clientes que mantienen el servicio pagan una cuota de mas de 60 mensual. Si se compara dicho valor con el valor minimo y maximo nos damos cuenta que es una tarifa alta.

> Muchos de estos clientes acumulan gastos totales grandes.

> Diariamente, la mayoria de los clientes generan costos de 0.05. Pero tambien hay ciertos clientes con costos diarios de mas de 3.

Analicemos las variables numericas de los clientes que cancelan.

In [36]:
# estadistica de clientes que cancelan el servicio
df_num[df_num['cancelacion'] == 1].describe()

,meses_contrato,cuenta_mensual,cuentas_total,cuentas_diarias,cancelacion
count,1869.000000,1869.000000,1869.000000,1869.000000,1869.0
mean,17.979133,74.441332,1531.796094,0.637061,1.0
std,19.531123,24.666053,1890.822994,0.808238,0.0
min,1.000000,18.850000,18.850000,0.010601,1.0
25%,2.000000,56.150000,134.500000,0.090000,1.0
50%,10.000000,79.650000,703.550000,0.249306,1.0
75%,29.000000,94.200000,2331.300000,0.817500,1.0
max,72.000000,118.350000,8684.800000,3.415000,1.0


> La mitad de los clientes que cancelan el servicio lo hacen a los 10 meses (antes del primer año) mucho antes de aquellos que mantienen el servicio (38 meses)

> La mitad de los clientes que cancelan, pagan 79.65 por mes mucho mas que aquellos que mantienen el servicio (64.45).

> Los clientes que cancelan acumulan menos gasto total lo cual es logico ya que tienen contratos mas cortos.

> La mitad de los clientes que cancelan tienen un gasto diario mucho mayor que aquellos que mantienen el servicio (0.24 >> 0.05)

### Analisis estadistico variables categoricas

Para las variables categoricas, seleccionamos los registros de los cliente que cancelan y obtenemos el conteo de cada valor.

In [37]:
# dataframe de variables categoricas de clientes que cancelan
df_bin_1 = df_bin[df_bin['cancelacion'] == 1]
df_bin_1.head()

,cancelacion,genero,jubilado,en_pareja,dependientes,servicio_telefonico,servicio_internet,seguridad_online,respaldo_online,proteccion_dispositivo,soporte_efectivo,Streaming_TV,Streaming_peliculas,tipo_contrato,facturacion_electronica,metodo_pago
2,1,1,0,0,0,1,1,1,1,2,1,1,1,0,1,Electronic check
3,1,1,1,1,0,1,1,1,2,2,1,2,2,0,1,Electronic check
4,1,0,1,1,0,1,1,1,1,1,2,2,1,0,1,Mailed check
18,1,1,1,0,0,1,1,2,1,2,1,1,2,1,0,Credit card (automatic)
19,1,1,1,0,0,0,1,1,1,1,1,1,1,0,1,Electronic check


In [38]:
# conteo de valores de cada variable categorica
for col in df_bin_1.columns:
    print('-'*10)
    print(col)
    print(df_bin_1[col].value_counts(normalize=True))

----------
cancelacion
1    1.0
Name: cancelacion, dtype: float64
----------
genero
0    0.502408
1    0.497592
Name: genero, dtype: float64
----------
jubilado
0    0.745318
1    0.254682
Name: jubilado, dtype: float64
----------
en_pareja
0    0.642055
1    0.357945
Name: en_pareja, dtype: float64
----------
dependientes
0    0.825575
1    0.174425
Name: dependientes, dtype: float64
----------
servicio_telefonico
1    0.909042
0    0.090958
Name: servicio_telefonico, dtype: float64
----------
servicio_internet
1    0.93954
0    0.06046
Name: servicio_internet, dtype: float64
----------
seguridad_online
1    0.781701
2    0.157838
0    0.060460
Name: seguridad_online, dtype: float64
----------
respaldo_online
1    0.659711
2    0.279829
0    0.060460
Name: respaldo_online, dtype: float64
----------
proteccion_dispositivo
1    0.64794
2    0.29160
0    0.06046
Name: proteccion_dispositivo, dtype: float64
----------
soporte_efectivo
1    0.773676
2    0.165864
0    0.060460
Name: soport

> Hay 1869 clientes que cancelan el servicio con la empresa.

> Presentan similares proporciones entre hombre y mujeres.

> 74% no son jubilados.

> La mayoria son solteros (64%).

> El 82% no poseen dependientes.

> El 90% cuenta con servicio telefonico.

> El 93% cuenta con servicion de internet.

> El 78% no cuenta con seguridad en linea.

> La mayoria tampoco no cuenta con respaldo en linea (66%)

> La mayoria no esta suscrito a proteccion del dispositivo (65%)

> La mayoria no cuentan con un soporte mejorado (77%)

> La proporcion entre los que cuentan con servicio de TV por cable y los que no es muy similar. Tambien ocurre lo mismo con el servicio de streaming de peliculas.

> El 88% tienen un contrato de tipo mes a mes.

> La mayoria prefiere recibir la factura electronica.

> El metodo de pago preferido por este grupo es Cheque Electronico.



Ahora, procedemos a obtener la misma estadistica para clientes que mantienen el servicio con la empresa.

In [39]:
# dataframe de variables categoricas de clientes que cancelan
df_bin_0 = df_bin[df_bin['cancelacion'] == 0]
df_bin_0.head()

,cancelacion,genero,jubilado,en_pareja,dependientes,servicio_telefonico,servicio_internet,seguridad_online,respaldo_online,proteccion_dispositivo,soporte_efectivo,Streaming_TV,Streaming_peliculas,tipo_contrato,facturacion_electronica,metodo_pago
0,0,0,0,1,1,1,1,1,2,1,2,2,1,1,1,Mailed check
1,0,1,0,0,0,1,1,1,1,1,1,1,2,0,0,Mailed check
5,0,0,0,0,1,1,1,1,1,1,2,2,2,0,1,Credit card (automatic)
6,0,0,1,1,0,1,1,2,2,2,2,2,2,2,1,Bank transfer (automatic)
7,0,1,0,1,0,1,1,2,1,1,2,1,1,2,1,Credit card (automatic)


In [40]:
# conteo de valores de cada variable categorica
for col in df_bin_0.columns:
    print('-'*10)
    print(col)
    print(df_bin_0[col].value_counts(normalize=True))

----------
cancelacion
0    1.0
Name: cancelacion, dtype: float64
----------
genero
1    0.507263
0    0.492737
Name: genero, dtype: float64
----------
jubilado
0    0.871005
1    0.128995
Name: jubilado, dtype: float64
----------
en_pareja
1    0.5276
0    0.4724
Name: en_pareja, dtype: float64
----------
dependientes
0    0.656595
1    0.343405
Name: dependientes, dtype: float64
----------
servicio_telefonico
1    0.90122
0    0.09878
Name: servicio_telefonico, dtype: float64
----------
servicio_internet
1    0.727484
0    0.272516
Name: servicio_internet, dtype: float64
----------
seguridad_online
1    0.394344
2    0.333140
0    0.272516
Name: seguridad_online, dtype: float64
----------
respaldo_online
2    0.368390
1    0.359094
0    0.272516
Name: respaldo_online, dtype: float64
----------
proteccion_dispositivo
1    0.364710
2    0.362774
0    0.272516
Name: proteccion_dispositivo, dtype: float64
----------
soporte_efectivo
1    0.392408
2    0.335077
0    0.272516
Name: soporte

> La proporcion en cuanto al genero son similares.

> La mayoria no son jubilados.

> La proporcion entre solteros y en pareja son similares.

> El 65% no tiene dependientes.

> La mayoria tiene servicio telefonico (90%)

> El 72% tiene servicio de internet.

> La cantidad de clientes que no tienen seguridad en linea es ligeramente mayor de los que si tienen (39%-33%)

> La proporcion de los que tienen respaldo online de los que no es muy similar. Lo mismo ocurre para proteccion del dispositivo, Streaming TV y streaming de peliculas.

> La cantidad de clientes que no tienen soporte mejorado es ligeramente mayor de los que si tiene (39%-33%)

> La mayoria tienen un tipo de contrato mes a mes.

> El 53% prefieren factura electronica.

> La proporcion en cuanto a los metodos de pagos son muy similares.

Para entender visualmente las conclusiones obtenidas a partir de la estadistica descriptiva, vamos a crear graficos tanto para las variables numericas como para las categoricas.

### Visualizacion variables numericas

In [42]:
df_num[df_num['cancelacion'] == 1]['meses_contrato'].median()

10.0

In [43]:
df_num[df_num['cancelacion'] == 0]['meses_contrato'].median()

38.0

In [50]:
cols = ['cuenta_mensual', 'cuentas_total', 'cuentas_diarias']


def resumen_estadistico(df, columnas):
    
    stats = []

    for col in columnas:
        cancelan = df[df['cancelacion'] == 1][col]
        no_cancelan = df[df['cancelacion'] == 0][col]

        stats.append(['Media', col, 'Cancelaron', cancelan.mean()])
        stats.append(['Media', col, 'No cancelaron', no_cancelan.mean()])
        stats.append(['Mediana', col, 'Cancelaron', cancelan.median()])
        stats.append(['Mediana', col, 'No cancelaron', no_cancelan.median()])

    new_df = pd.DataFrame(stats, columns=['estadistico', 'variable', 'grupo', 'valor'])
    return new_df

In [51]:
df_resumen = resumen_estadistico(df_num, cols)

In [52]:
df_resumen

,estadistico,variable,grupo,valor
0,Media,cuenta_mensual,Cancelaron,74.441332
1,Media,cuenta_mensual,No cancelaron,61.307408
2,Mediana,cuenta_mensual,Cancelaron,79.650000
3,Mediana,cuenta_mensual,No cancelaron,64.450000
4,Media,cuentas_total,Cancelaron,1531.796094
5,Media,cuentas_total,No cancelaron,2555.344141
6,Mediana,cuentas_total,Cancelaron,703.550000
7,Mediana,cuentas_total,No cancelaron,1683.600000
8,Media,cuentas_diarias,Cancelaron,0.637061
9,Media,cuentas_diarias,No cancelaron,0.160736
